In [0]:
!pip install umap-learn numpy hdbscan bertopic langchain databricks-sdk[openai]

In [0]:
import numpy as np

CATALOG = "kohei_arai"
SCHEMA = "demo"
WRITEBACK_TABLE = "calls_log_silver_index_writeback_table"
PLOT_TABLE = "calls_log_plot_table"
LABELS_TABLE = "calls_log_labels_table"
GOLD_TABLE = "calls_log_gold"

In [0]:
plotDF = spark.table(f"{CATALOG}.{SCHEMA}.{PLOT_TABLE}")
plot_df = plotDF.toPandas()
display(plot_df)

In [0]:
# Install the required packages
# %pip install databricks-sdk[openai] mlflow

from tqdm import tqdm
import numpy as np
import pandas as pd
from databricks.sdk import WorkspaceClient

def labelling(config, plot_df):

    results = pd.DataFrame()

    sample_size = config['labelling']['sample_size']
    prompt = config['labelling']['prompt']
    model = config['labelling']['model']

    question = config['question']
    cluster_ids = plot_df['cluster'].unique()

    for _, cluster_id in tqdm(enumerate(cluster_ids), total=len(cluster_ids)):
        args_sample = plot_df[plot_df['cluster'] == cluster_id]['text'].values
        args_sample = np.random.choice(args_sample, size=min(len(args_sample), sample_size), replace=False)

        args_sample_outside = plot_df[plot_df['cluster'] != cluster_id]['text'].values
        args_sample_outside = np.random.choice(args_sample_outside, size=min(len(args_sample_outside), sample_size), replace=False)

        label = generate_label(question, args_sample, args_sample_outside, prompt, model)
        results = pd.concat([results, pd.DataFrame([{'cluster-id': cluster_id, 'label': label}])], ignore_index=True)

    return results

def generate_label(question, args_sample, args_sample_outside, prompt, model):
    w = WorkspaceClient()
    llm = w.serving_endpoints.get_open_ai_client()
    outside = '\n * ' + '\n * '.join(args_sample_outside)
    inside = '\n * ' + '\n * '.join(args_sample)
    input_text = f"Question of the consultation: {question}\n\n" + \
                 f"Examples of arguments OUTSIDE the cluster:\n{outside}\n\n" + \
                 f"Examples of arguments INSIDE the cluster:\n{inside}"
    response = llm.chat.completions.create(
        messages=[{"role": "system", "content": prompt}, {"role": "user", "content": input_text}],
        model=model,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()

# Config example
config = {
    'output_dir': 'your_output_directory',
    'labelling': {
        'sample_size': 5,
        'prompt':  """あなたは、コンサルテーションの議論を分類するラベリングアシスタントです。ある議論のまとまり（クラスター）に対して、その内容を端的に表すラベルを1つ作成してください。クラスターの主な話題、関連する問い合わせのリスト、関係ない問い合わせのリストが与えられます。ラベルは以下のルールに従ってください：
        ・質問からすでに明らかな文脈は含めない
        ・「〜に関すること」「〜についての問い合わせ」「〜を行う」といった表現は使わない
        ・名詞で終わるのではなく、「〜の変更」「〜の依頼」など動詞を体言止めで終える
        ・簡潔であること
        ・クラスター内と外の問い合わせを正しく区別できるよう、十分な正確さをもつこと
回答はラベルのみを返してください。""",
        'model': 'databricks-meta-llama-3-3-70b-instruct'
    },
    'question': 'これらの問い合わせの主なテーマは何か?'
}

# Run labelling
res = labelling(config, plot_df)
display(res)

In [0]:
spark.createDataFrame(res).write.mode("overwrite").saveAsTable(
    f"{CATALOG}.{SCHEMA}.{LABELS_TABLE}"
)

In [0]:
# Set the catalog and schema
spark.sql(f"USE CATALOG `{CATALOG}`")
spark.sql(f"USE SCHEMA `{SCHEMA}`")

# Create or replace the gold table by joining plot and labels tables
query = f"""
CREATE OR REPLACE TABLE {GOLD_TABLE} AS 
SELECT
  a.x,
  a.y,
  a.text,
  b.label
FROM
  {PLOT_TABLE} a
  LEFT JOIN {LABELS_TABLE} b 
  ON a.cluster = b.`cluster-id`
"""

spark.sql(query)